In [2]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [4]:
# Setup vector store
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
# )


client = weaviate.connect_to_local(
                     host="192.168.100.125",
                        port=8080,
                        grpc_port=50051
                )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [5]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [6]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=10,
    alpha=0.3,
)

In [7]:
question = "ABC"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : general mức phạt : none nội_dung vi_phạm : luật giao_thông quy_định bằng lái_xe hạng a1 : cấp cho người lái_xe mô_tô hai bánh có dung_tích xi - lanh đến 125 phân_khối hoặc có công_suất động_cơ_điện đến 11 kw . xe mô_tô hai bánh , dung_tích xi - lanh từ 50 - dưới 175 phân_khối , người khuyết_tật điều_khiển xe mô_tô ba bánh dùng cho người khuyết_tật được cấp giấy_phép lái_xe hạng a1 .

Metadata: {'category': 'General', 'fine_amount': 'None', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: General\nMức phạt: None\nNội dung vi phạm: Luật giao thông quy định Bằng lái xe hạng A1: cấp cho người lái xe mô tô hai bánh có dung tích xi-lanh đến 125 phân khối hoặc có công suất động cơ điện đến 11 kW. Xe mô tô hai bánh, dung tích xi-lanh từ 50 - dưới 175 phân khối, người khuyết tật điều khiển xe mô tô ba bánh dùng cho người khuyết tật được cấp giấy phép lái xe hạng A1.'}

Score: 0.30000001192092896

Result 2:

Text: loại phương_tiện : general mức ph

In [8]:
# client.collections.delete_all()

In [9]:
def classify_query(question: str) -> str:
        # Keywords indicating violation/penalty questions
        violation_keywords = [
            "xử phạt", "bị phạt", "bị gì", "xử lý", "nộp phạt", 
            "phạt tiền", "phạt bao nhiêu", "mức phạt", "tước giấy phép",
            "trừ điểm", "phạt như thế nào", "bị phạt gì", "hình thức xử phạt",
            "xử phạt hành chính", "tiền phạt", "phạt hành chính", "bị tịch thu",
            "thu giữ", "tạm giữ"
        ]
        
        question_lower = question.lower()
        
        # Check if any violation keywords are in the question
        for keyword in violation_keywords:
            if keyword in question_lower:
                return "violation_penalty"
        
    
        return "general_knowledge"